In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os
import itertools # For creating combinations of MLP dims

# MAPE를 위한 유틸리티 함수 (0으로 나누는 오류 방지)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_true = y_true != 0
    if np.sum(non_zero_true) == 0:
        return 0.0 # 모든 y_true가 0인 경우 MAPE는 0으로 처리
    return np.mean(np.abs((y_true[non_zero_true] - y_pred[non_zero_true]) / y_true[non_zero_true])) * 100

#파일 로드
df = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 필요한 컬럼 추출
df_processed = df[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# user_id와 business_id를 연속적인 정수 ID로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

df_processed.loc[:, 'user_encoded'] = user_encoder.fit_transform(df_processed['user_id'])
df_processed.loc[:, 'business_encoded'] = business_encoder.fit_transform(df_processed['business_id'])

num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)

# 데이터 분할
# 논문에서 제시된 70/10/20 비율로 데이터 분할
train_val_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)
val_size_ratio = 1 / 8 # 10% of total data (1/8 of 80%)
train_df, val_df = train_test_split(train_val_df, test_size=val_size_ratio, random_state=42)

print(f"전체 데이터 수: {len(df_processed)}")
print(f"학습 데이터 수: {len(train_df)} ({len(train_df)/len(df_processed)*100:.2f}%)")
print(f"검증 데이터 수: {len(val_df)} ({len(val_df)/len(df_processed)*100:.2f}%)")
print(f"테스트 데이터 수: {len(test_df)} ({len(test_df)/len(df_processed)*100:.2f}%)")

# --- PyTorch Dataset 및 DataLoader 정의 ---
class ReviewDataset(Dataset):
    def __init__(self, df):
        self.user_ids = torch.tensor(df['user_encoded'].values, dtype=torch.long)
        self.business_ids = torch.tensor(df['business_encoded'].values, dtype=torch.long)
        # sentiment_vector가 비어있을 수 있으므로 numpy 배열로 변환 후 tolist()
        self.sentiment_vectors = torch.tensor(np.array(df['sentiment_vector'].tolist()), dtype=torch.float)
        self.stars = torch.tensor(df['stars'].values, dtype=torch.float)

    def __len__(self):
        return len(self.stars)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.business_ids[idx], self.sentiment_vectors[idx], self.stars[idx]

# --- 모델 아키텍처 정의 ---
class CustomerRestaurantInteractionModule(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim, mlp_dims):
        super(CustomerRestaurantInteractionModule, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.business_embedding = nn.Embedding(num_businesses, embedding_dim)
        
        layers = []
        # 논문에 따라 MLP의 첫 입력은 임베딩 결합 벡터 (embedding_dim * 2)
        input_dim = embedding_dim * 2
        for dim in mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        # MLP가 없을 경우를 대비하여 output_dim 설정 (최종 레이어 차원)
        self.output_dim = mlp_dims[-1] if mlp_dims else embedding_dim * 2

    def forward(self, user_ids, business_ids):
        user_vec = self.user_embedding(user_ids)
        business_vec = self.business_embedding(business_ids)
        combined_vec = torch.cat((user_vec, business_vec), dim=1)
        interaction_features = self.mlp(combined_vec)
        return interaction_features

class ReviewAspectModule(nn.Module):
    # self-attention 및 키워드 추출 부분은 'sentiment_vector'에 이미 반영되었다고 가정
    # 따라서, sentiment_vector_dim이 키워드 특성의 차원이라고 볼 수 있음
    def __init__(self, sentiment_vector_dim, aspect_mlp_dims):
        super(ReviewAspectModule, self).__init__()
        layers = []
        input_dim = sentiment_vector_dim
        for dim in aspect_mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*layers)
        self.output_dim = aspect_mlp_dims[-1] if aspect_mlp_dims else sentiment_vector_dim

    def forward(self, sentiment_vectors):
        aspect_features = self.mlp(sentiment_vectors)
        return aspect_features

class AATRec(nn.Module):
    def __init__(self, num_users, num_businesses, embedding_dim,
                 user_biz_mlp_dims, aspect_mlp_dims, final_mlp_dims,
                 sentiment_vector_dim):
        super(AATRec, self).__init__()
        self.customer_restaurant_interaction_module = CustomerRestaurantInteractionModule(
            num_users, num_businesses, embedding_dim, user_biz_mlp_dims
        )
        # sentiment_vector_dim은 실제 sentiment_vector의 차원을 가져옴
        self.review_aspect_module = ReviewAspectModule(
            sentiment_vector_dim, aspect_mlp_dims
        )
        
        # 평점 예측 모듈의 입력 차원 계산
        final_input_dim = self.customer_restaurant_interaction_module.output_dim + \
                          self.review_aspect_module.output_dim
        
        layers = []
        input_dim = final_input_dim
        for dim in final_mlp_dims:
            layers.append(nn.Linear(input_dim, dim))
            layers.append(nn.ReLU())
            input_dim = dim
        layers.append(nn.Linear(input_dim, 1)) # 최종 출력은 평점 (1차원)
        self.prediction_mlp = nn.Sequential(*layers)

    def forward(self, user_ids, business_ids, sentiment_vectors):
        user_biz_features = self.customer_restaurant_interaction_module(user_ids, business_ids)
        aspect_features = self.review_aspect_module(sentiment_vectors)
        combined_features = torch.cat((user_biz_features, aspect_features), dim=1)
        predicted_rating = self.prediction_mlp(combined_features)
        return predicted_rating.squeeze() # 1차원 평점 반환

# sentiment_vector의 실제 차원을 데이터프레임에서 동적으로 가져옴
sentiment_vector_dim = len(df_processed['sentiment_vector'].iloc[0]) if not df_processed.empty else 15 # 데이터 없으면 기본값

# --- 데이터셋 및 DataLoader 생성 (여기로 이동!) ---
train_dataset = ReviewDataset(train_df)
val_dataset = ReviewDataset(val_df)
test_dataset = ReviewDataset(test_df) # test_loader는 최종 평가에만 사용되므로 여기에 포함

# --- 하이퍼파라미터 탐색 공간 정의 ---
# 논문에서 제시된 값을 기준으로, 주변 값을 탐색
param_grid = {
    'embedding_dim': [64],
    'learning_rate': [0.001, 0.0005, 0.0001], # 논문 값 포함하여 주변 탐색
    'batch_size': [128, 256, 512], # 논문 값 포함하여 주변 탐색
    'user_biz_mlp_hidden_dims': [[128, 64], [64, 32]], # 1번 모듈 MLP 히든 레이어 차원 조합 (출력 차원 자동 결정)
    'aspect_mlp_hidden_dims': [[64, 32], [32, 16]], # 2번 모듈 MLP 히든 레이어 차원 조합
    'final_mlp_hidden_dims': [[64, 32], [32, 16]] # 최종 평점 예측 모듈 MLP 히든 레이어 차원 조합
}

best_overall_rmse = float('inf')
best_params = None
results = []

# 하이퍼파라미터 그리드를 순회
for i, params in enumerate(ParameterGrid(param_grid)):
    #print(f"\n실험 {i+1}/{len(ParameterGrid(param_grid))}")
    #print(f"현재 파라미터: {params}")

    # 하이퍼파라미터 설정
    embedding_dim = params['embedding_dim']
    learning_rate = params['learning_rate']
    batch_size = params['batch_size']
    user_biz_mlp_dims = params['user_biz_mlp_hidden_dims']
    aspect_mlp_dims = params['aspect_mlp_hidden_dims']
    final_mlp_dims = params['final_mlp_hidden_dims']
    
    epochs = 50 # 각 조합별 최대 에폭
    patience = 5 # 조기 종료를 위한 대기 에폭
    min_delta = 0.001 # 성능 개선을 판단하는 최소 변화량
    
    current_best_val_rmse = float('inf')
    epochs_no_improve = 0
    
    model_save_path = f'temp_best_model_{i}.pt' # 각 조합별 모델 저장 경로

    # DataLoader 생성 (batch_size에 따라 새로 생성)
    # train_loader와 val_loader는 여기서 새로 생성되어야 합니다.
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    # test_loader는 최종 평가에만 사용되므로 여기서는 생성하지 않아도 됩니다.

    # 모델 인스턴스 생성 및 초기화
    model = AATRec(num_users, num_businesses, embedding_dim,
                   user_biz_mlp_dims, aspect_mlp_dims, final_mlp_dims,
                   sentiment_vector_dim)
    
    # 손실 함수 및 옵티마이저
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # --- 학습 루프 (조기 종료 포함) ---
    for epoch in range(epochs):
        # 학습 단계
        model.train()
        total_train_loss = 0
        for user_ids, business_ids, sentiment_vectors, stars in train_loader:
            optimizer.zero_grad()
            predictions = model(user_ids, business_ids, sentiment_vectors)
            loss = criterion(predictions, stars)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        # 검증 단계
        model.eval()
        total_val_loss = 0
        val_predictions = []
        val_true_ratings = []
        with torch.no_grad():
            for user_ids, business_ids, sentiment_vectors, stars in val_loader:
                predictions = model(user_ids, business_ids, sentiment_vectors)
                loss = criterion(predictions, stars)
                total_val_loss += loss.item()
                val_predictions.extend(predictions.tolist())
                val_true_ratings.extend(stars.tolist())

        current_val_rmse = np.sqrt(mean_squared_error(val_true_ratings, val_predictions))

        # print(f"Epoch {epoch+1}/{epochs}, "
        #       f"Train Loss: {total_train_loss / len(train_loader):.4f}, "
        #       f"Val Loss: {total_val_loss / len(val_loader):.4f}, "
        #       f"Val RMSE: {current_val_rmse:.4f}")

        # 조기 종료 로직
        if current_val_rmse < current_best_val_rmse - min_delta:
            current_best_val_rmse = current_val_rmse
            epochs_no_improve = 0
            torch.save(model.state_dict(), model_save_path) # 현재 조합의 최적 모델 저장
            # print(f"  --> 검증 RMSE 개선됨. 현재 조합의 최적 모델 저장됨: {current_best_val_rmse:.4f}")
        else:
            epochs_no_improve += 1
            # print(f"  --> 검증 RMSE 개선되지 않음. 대기 중... ({epochs_no_improve}/{patience})")
            if epochs_no_improve == patience:
                #print(f"조기 종료! {patience} epoch 동안 RMSE 개선이 없었음")
                break
    
    # 각 조합별 결과 기록
    results.append({
        'params': params,
        'val_rmse': current_best_val_rmse
    })

    # 전체 최적 파라미터 업데이트
    if current_best_val_rmse < best_overall_rmse:
        best_overall_rmse = current_best_val_rmse
        best_params = params
        # 최종 최적 모델 가중치는 이 시점에서 저장하지 않고, 최종 테스트 전에 로드할 temp_best_model_X.pt로 간주
        # 또는 best_params를 사용하여 다시 학습시켜 저장 가능

print(f"최적 검증 RMSE: {best_overall_rmse:.4f}")
print(f"최적 파라미터: {best_params}")

# --- 최종 모델 학습 및 테스트 (최적 파라미터 사용) ---
if best_params:
    print("\n 최적 파라미터로 학습")
    final_embedding_dim = best_params['embedding_dim']
    final_learning_rate = best_params['learning_rate']
    final_batch_size = best_params['batch_size']
    final_user_biz_mlp_dims = best_params['user_biz_mlp_hidden_dims']
    final_aspect_mlp_dims = best_params['aspect_mlp_hidden_dims']
    final_final_mlp_dims = best_params['final_mlp_hidden_dims']

    final_model = AATRec(num_users, num_businesses, final_embedding_dim,
                         final_user_biz_mlp_dims, final_aspect_mlp_dims, final_final_mlp_dims,
                         sentiment_vector_dim)
    final_criterion = nn.MSELoss()
    final_optimizer = optim.Adam(final_model.parameters(), lr=final_learning_rate)

    final_train_loader = DataLoader(train_dataset, batch_size=final_batch_size, shuffle=True)
    final_val_loader = DataLoader(val_dataset, batch_size=final_batch_size, shuffle=False)
    final_test_loader = DataLoader(test_dataset, batch_size=final_batch_size, shuffle=False)
    
    final_epochs = 50 # 충분한 에폭
    final_patience = 10 # 조기 종료 대기 에폭을 더 길게 설정할 수 있음
    final_min_delta = 0.0005 # 더 엄격한 개선 기준
    
    best_final_val_rmse = float('inf')
    epochs_no_improve_final = 0
    final_model_path = 'final_best_aat_rec_model.pt'

    for epoch in range(final_epochs):
        # 학습 단계
        final_model.train()
        total_train_loss = 0
        for user_ids, business_ids, sentiment_vectors, stars in final_train_loader:
            final_optimizer.zero_grad()
            predictions = final_model(user_ids, business_ids, sentiment_vectors)
            loss = final_criterion(predictions, stars)
            loss.backward()
            final_optimizer.step()
            total_train_loss += loss.item()

        # 검증 단계
        final_model.eval()
        total_val_loss = 0
        val_predictions = []
        val_true_ratings = []
        with torch.no_grad():
            for user_ids, business_ids, sentiment_vectors, stars in final_val_loader:
                predictions = final_model(user_ids, business_ids, sentiment_vectors)
                loss = final_criterion(predictions, stars)
                total_val_loss += loss.item()
                val_predictions.extend(predictions.tolist())
                val_true_ratings.extend(stars.tolist())
        
        current_val_rmse = np.sqrt(mean_squared_error(val_true_ratings, val_predictions))

        print(f"Final Train Epoch {epoch+1}/{final_epochs}, "
              f"Train Loss: {total_train_loss / len(final_train_loader):.4f}, "
              f"Val Loss: {total_val_loss / len(final_val_loader):.4f}, "
              f"Val RMSE: {current_val_rmse:.4f}")

        # 조기 종료 로직
        if current_val_rmse < best_final_val_rmse - final_min_delta:
            best_final_val_rmse = current_val_rmse
            epochs_no_improve_final = 0
            torch.save(final_model.state_dict(), final_model_path)
            print(f"RMSE 개선됨. 모델 저장됨: {best_final_val_rmse:.4f}")
        else:
            epochs_no_improve_final += 1
            print(f"RMSE 개선되지 않음. ({epochs_no_improve_final}/{final_patience})")
            if epochs_no_improve_final == final_patience:
                print(f"조기 종료 - {final_patience} epoch 동안 검증 RMSE 개선이 없었음")
                break

    # --- 최종 모델 테스트 ---
    if os.path.exists(final_model_path):
        final_model.load_state_dict(torch.load(final_model_path))
        print(f"최적 모델 가중치 {final_model_path}")
    else:
        print(f"최적의 최종 모델 가중치 '{final_model_path}'를 찾을 수 없습니다. 현재 모델 상태로 테스트를 진행합니다.")

    final_model.eval()
    test_predictions = []
    true_ratings = []

    with torch.no_grad():
        for user_ids, business_ids, sentiment_vectors, stars in final_test_loader:
            predictions = final_model(user_ids, business_ids, sentiment_vectors)
            test_predictions.extend(predictions.tolist())
            true_ratings.extend(stars.tolist())

    mse = mean_squared_error(true_ratings, test_predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(true_ratings, test_predictions)
    mape = mean_absolute_percentage_error(true_ratings, test_predictions)

    print(f"\n성능 평가 (최적 파라미터)")
    print(f"Selected Hyperparameters: {best_params}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
else:
    print("최적 파라미터를 찾지 못해 최종 모델 학습 및 테스트를 건너뜁니다.")

전체 데이터 수: 447796
학습 데이터 수: 313456 (70.00%)
검증 데이터 수: 44780 (10.00%)
테스트 데이터 수: 89560 (20.00%)
최적 검증 RMSE: 0.6879
최적 파라미터: {'aspect_mlp_hidden_dims': [64, 32], 'batch_size': 128, 'embedding_dim': 64, 'final_mlp_hidden_dims': [32, 16], 'learning_rate': 0.001, 'user_biz_mlp_hidden_dims': [128, 64]}

 최적 파라미터로 학습
Final Train Epoch 1/50, Train Loss: 0.6888, Val Loss: 0.5094, Val RMSE: 0.7137
RMSE 개선됨. 모델 저장됨: 0.7137
Final Train Epoch 2/50, Train Loss: 0.4747, Val Loss: 0.4860, Val RMSE: 0.6971
RMSE 개선됨. 모델 저장됨: 0.6971
Final Train Epoch 3/50, Train Loss: 0.4398, Val Loss: 0.4741, Val RMSE: 0.6885
RMSE 개선됨. 모델 저장됨: 0.6885
Final Train Epoch 4/50, Train Loss: 0.4129, Val Loss: 0.4892, Val RMSE: 0.6994
RMSE 개선되지 않음. (1/10)
Final Train Epoch 5/50, Train Loss: 0.3884, Val Loss: 0.4802, Val RMSE: 0.6929
RMSE 개선되지 않음. (2/10)
Final Train Epoch 6/50, Train Loss: 0.3660, Val Loss: 0.4909, Val RMSE: 0.7006
RMSE 개선되지 않음. (3/10)
Final Train Epoch 7/50, Train Loss: 0.3420, Val Loss: 0.5080, Val RMSE: 0.712